# Chapter 1 - Stroke width, colors, opacities, layers and positioning
**Latest update : 6.6.2021**  
After reading this chapter, you will be able to build up Mobjects on scenes, no animations included yet.  
As python is very vertyle, there multiple ways to acomplish a certain task, but some options are not "best practice".
Here, I want to show the best practive (labeled with **BEST** and the **green check with the star**), other possible options (labeled with **YES** and the **green check**), and options that do not work (labeled with **NO** and the **red cross**)  
Please ignore the next three cells, they are only for setting up the example scene.

In [ ]:
from manim import *

In [ ]:
YES = SVGMobject("check.svg").to_edge(LEFT, buff=1)
BEST = YES.copy()
BEST.add(Star(color= YELLOW, fill_opacity=1).scale(0.5).move_to(BEST).shift(0.5*DOWN+0.5*RIGHT))
NO  = Cross(Square(), stroke_color = RED_D, stroke_width = 38).scale(0.9).move_to(YES)

In [ ]:
params = "-v WARNING -r 500,100 -s  --disable_caching Example"

## Stoke width :

strokes can be set in multiple ways:  
The recomendend ways are via  `Circle(stroke_width=30)` , `c.set_stroke(width = 30)` or `c.set_style(stroke_width= 30)`  
Also possible, but not the best solution is `c.stroke_width = 30` and `c.set(stroke_width = 30)`  
Also possible, but not recommended because deprecated is `c.set_stroke_width(30)`  

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle(stroke_width=30)
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle()
        c.set_stroke(width = 30)
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle()
        c.set_style(stroke_width= 30)
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle()
        c.stroke_width = 30
        self.add(YES,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle()
        c.set(stroke_width = 30)
        self.add(YES,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Circle()
        c.set_stroke_width(30)
        self.add(NO,c)

## Colors and Opacity:
* Color parameters for Mobjects are `stroke_color`, `fill_color` and `color`.   The paramter  `color` automatically sets both `stroke_color` and `fill_color`.  
The recomendend ways to set **colors** are via  `c = Circle(fill_color= BLUE, fill_opacity= 1 )` , `c.set_fill(color=RED)` or `c.set_style(fill_color=GREEN)`  
Not possible are `c.fill_color=YELLOW` ,   `c.set(fill_color=YELLOW)` and `c.set_fill_color(YELLOW)`


* Opacity paramters are  for Mobjects are `fill_opacity` and `stroke_opacity` (there is **not**  `opacity` here).  
The recomendend ways to set **opacites** are via  `c = Circle(fill_color= BLUE, fill_opacity= 0.5 )` , `c.set_fill(color=RED)` or `c.set_style(fill_color=GREEN)`  

Analog to colors, `c.fill_opacity=1`, `c.set(fill_opacity=1)` and `c.set_fill_opacity(1)` are not possible. (to keep things short, these examples are not shown).


### Colors

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_color= BLUE, fill_opacity= 1 )
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_color= BLUE, fill_opacity= 1)
        c.set_fill(color=RED)
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_color= BLUE, fill_opacity= 1)
        c.set_style(fill_color=GREEN)
        self.add(BEST,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_opacity= 1)
        c.fill_color=YELLOW
        self.add(NO,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_opacity= 1)
        c.set(fill_color=YELLOW)
        self.add(NO,c)

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Square(fill_opacity= 1)
        c.set_fill_color(YELLOW)
        self.add(NO,c)

### Opacities

In [ ]:
%%manim $params
class Example(Scene):
    def construct(self):
        c = Ellipse(color= YELLOW, fill_opacity=0.3).scale(2)
        self.add(BEST,c)

In [ ]:
%%manim  $params
class Example(Scene):
    def construct(self):
        c = Ellipse(color= YELLOW).scale(2)
        c.set_fill(opacity=0.5) # be careful: here, it must be `opacity` and not `fill_opacity` !
        self.add(BEST,c)

In [ ]:
%%manim  $params
class Example(Scene):
    def construct(self):
        c = Ellipse(color= YELLOW).scale(2)
        c.set_style(fill_opacity=0.7) # and here, it must be `fill_opacity` and not `opacity` !
        self.add(BEST,c)

## Layers